In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Github/

/content/drive/MyDrive/Github


In [3]:
!git pull https://github.com/rohit-khoiwal-30/msgmt.git

remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 32 (delta 19), reused 10 (delta 4), pack-reused 0
Unpacking objects: 100% (32/32), done.
From https://github.com/rohit-khoiwal-30/msgmt
 * branch            HEAD       -> FETCH_HEAD
Updating a5224bf..01b7e22
Fast-forward
 .ipynb_checkpoints/preprocess-checkpoint.ipynb | 1003 +++
 augment_train.csv                              | 8205 ++++++++++++++++++++++++
 bertCONCT.ipynb                                |  963 +--
 bertEUCL.ipynb                                 |  870 +--
 bertTripleLoss.ipynb                           | 1058 +--
 preprocess.ipynb                               | 2063 +++---
 useCONCT -.ipynb                               |  366 --
 useCONCT.ipynb                                 |  611 ++
 useEUCL-checkpoint.ipynb                       |  518 --
 useEUCL.ipynb                                  |  834 ++-
 useTripleLoss .ipyn

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("augment_train.csv")
df_test = pd.read_csv("evaluation.csv")

In [6]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from keras.regularizers import l2

try:
    import tensorflow_hub as hub
except ModuleNotFoundError:
    %pip install tensorflow_hub
    import tensorflow_hub as hub
    
try:
    from keras_self_attention import SeqSelfAttention
except ModuleNotFoundError:
    %pip install -q keras-self-attention
    from keras_self_attention import SeqSelfAttention

import numpy as np


# %pip install -q transformers
# %pip install -q -U tensorflow-text
# %pip install -q tf-models-official==2.7.0

import math as m

In [7]:
huburl = "https://tfhub.dev/google/universal-sentence-encoder/4" 
fine_tuned_module_object = hub.load(huburl)

In [8]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    text_batch, reason_batch, nonreason_batch, y_hat = [], [], [], []
    all_groups = list(range(in_groups[0].shape[0]))

    group_idx = np.random.choice(all_groups, size = batch_halfsize*2)
    text_batch += [in_groups[0][c_idx] for c_idx in group_idx]
    
    b_group_idx = group_idx
    y_hat += [1]*batch_halfsize
    reason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
    

    non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
    b_group_idx = non_group_idx
    y_hat += [0]*batch_halfsize
    nonreason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
            
    return np.stack(text_batch, 0), np.stack(reason_batch, 0), np.stack(nonreason_batch, 0), np.stack(y_hat, 0)

def siam_gen(in_groups, batch_size = 32):
    while True:
        text_stack, reason_stack, nonreason_stack, y_hatstack = gen_random_batch(in_groups, batch_size//2)
        yield [text_stack, reason_stack, nonreason_stack], y_hatstack


def identity_loss(y_true, y_pred):
    return K.mean(y_pred)

def triplet_loss(x, alpha = 0.2):
    # Triplet Loss function.
    anchor,positive,negative = x
    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)
    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)
    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
    return loss

In [9]:
def accuracy(y_true, y_pred):
    results = (y_pred >= 0.5).astype(int).squeeze()
    return np.mean([y_true == results])

In [10]:
huburl = "https://tfhub.dev/google/universal-sentence-encoder/4" 
fine_tuned_module_object = hub.load(huburl)

In [45]:
def encoder1(name):
    inputs = Input(shape=(), dtype=tf.string)

    shared_embedding_layer = hub.KerasLayer(fine_tuned_module_object, trainable=True, name=name)
    embedding_output= shared_embedding_layer(inputs)
    a = tfl.Dense(128, activation="linear", kernel_regularizer=l2(1e-3))(embedding_output)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    outputs = tfl.Activation('relu')(a)
    # outputs = tfl.Lambda(lambda x: K.l2_normalize(x,axis=-1))(a)
    # a = tfl.BatchNormalization()(a)
    # outputs = tfl.Activation('relu')(a)
    
    model = Model(inputs, outputs)
    return model

def encoder2(name):
    inputs = Input(shape=(), dtype=tf.string)

    shared_embedding_layer = hub.KerasLayer(fine_tuned_module_object, trainable=True, name=name)
    embedding_output= shared_embedding_layer(inputs)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(embedding_output)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    outputs = tfl.Activation('relu')(a)
    # outputs = tfl.Lambda(lambda x: K.l2_normalize(x,axis=-1))(a)
    # a = tfl.BatchNormalization()(a)
    # outputs = tfl.Activation('relu')(a)
    
    model = Model(inputs, outputs)
    return model

In [46]:
def get_model():
    #textEncoder
    inputText = Input(shape=(), dtype=tf.string)
    textModel = encoder1("textBertEncoder")
    text_embedd = textModel(inputText)

    #reasonEncoder
    reasonModel = encoder2("reasonBertEncoder")

    inputReason = Input(shape=(), dtype=tf.string)
    reason_embedd = reasonModel(inputReason)

    #nonReasonEncoder
    inputNotReason = Input(shape=(), dtype=tf.string)
    nonreason_embedd = reasonModel(inputNotReason)

    loss = tfl.Lambda(triplet_loss)([text_embedd, reason_embedd, nonreason_embedd])

    model = Model(inputs = [inputText, inputReason, inputNotReason], outputs = [loss], name="bertModel")
    return model, textModel, reasonModel

In [59]:
model, textModel, reasonModel = get_model()

In [60]:
model.summary()

Model: "bertModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 input_24 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 input_25 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 model_4 (Functional)           (None, 32)           256874720   ['input_21[0][0]']               
                                                                                          

In [61]:
model.compile(optimizer="adam", loss = identity_loss)

In [62]:
eval_df = df_test[df_test['label'] == 1]

In [63]:
loss_history = model.fit(siam_gen([df["texts"], df["reason"]], 128), 
                         validation_data = siam_gen([eval_df["text"].values, eval_df["reason"].values], 128),
                         steps_per_epoch=50, validation_steps=10, epochs = 50, 
                         verbose = True, use_multiprocessing=True)

Epoch 1/50
50/50 [==============================] - 32s 496ms/step - loss: 2.3444 - val_loss: 0.7025
Epoch 2/50
50/50 [==============================] - 19s 379ms/step - loss: 1.2627 - val_loss: 0.9153
Epoch 3/50
50/50 [==============================] - 20s 396ms/step - loss: 0.9687 - val_loss: 0.9926
Epoch 4/50
50/50 [==============================] - 23s 471ms/step - loss: 0.8320 - val_loss: 0.9954
Epoch 5/50
50/50 [==============================] - 20s 399ms/step - loss: 0.7333 - val_loss: 1.0626
Epoch 6/50
50/50 [==============================] - 21s 420ms/step - loss: 0.6917 - val_loss: 1.4423
Epoch 7/50
50/50 [==============================] - 23s 469ms/step - loss: 0.6418 - val_loss: 1.5822
Epoch 8/50
50/50 [==============================] - 20s 402ms/step - loss: 0.6104 - val_loss: 1.6818
Epoch 9/50
50/50 [==============================] - 20s 410ms/step - loss: 0.5686 - val_loss: 1.3779
Epoch 10/50
50/50 [==============================] - 24s 479ms/step - loss: 0.5596 - val_lo

Process Keras_worker_ForkPoolWorker-78:
Process Keras_worker_ForkPoolWorker-74:
Process Keras_worker_ForkPoolWorker-84:
Process Keras_worker_ForkPoolWorker-81:
Process Keras_worker_ForkPoolWorker-80:
Process Keras_worker_ForkPoolWorker-76:
Process Keras_worker_ForkPoolWorker-71:
Process Keras_worker_ForkPoolWorker-85:
Process Keras_worker_ForkPoolWorker-66:
Process Keras_worker_ForkPoolWorker-75:
Process Keras_worker_ForkPoolWorker-64:
Process Keras_worker_ForkPoolWorker-72:
Process Keras_worker_ForkPoolWorker-67:
Process Keras_worker_ForkPoolWorker-69:
Process Keras_worker_ForkPoolWorker-68:
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-83:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-65:
Traceback (most recent call last):
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-70:
Process Keras_worker_ForkPoolWorker-77:
Process Keras_worker_ForkPoolW

KeyboardInterrupt: ignored

Process Keras_worker_ForkPoolWorker-79:
Process Keras_worker_ForkPoolWorker-82:
Process Keras_worker_ForkPoolWorker-63:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()


In [53]:
left_input = tf.keras.Input(shape=(), dtype=tf.string)
right_input = tf.keras.Input(shape=(), dtype=tf.string)

textEmbedding = textModel(left_input)
reasonEmbedding = reasonModel(right_input)

cosine_similiarity=tf.keras.layers.Dot(axes=-1,normalize=True)([textEmbedding, reasonEmbedding])


classifier_model = Model(inputs = [left_input, right_input], outputs = [cosine_similiarity], name="classifier_model")

In [54]:
classifier_model.summary()

Model: "classifier_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 model_2 (Functional)           (None, 32)           256874720   ['input_19[0][0]']               
                                                                                                  
 model_3 (Functional)           (None, 32)           256833120   ['input_20[0][0]']               
                                                                                   

In [55]:
y_preds = classifier_model.predict([df['texts'], df['reason']])
accuracy(df["label"], y_preds)

0.8532747896084888

In [56]:
y_preds = classifier_model.predict([df_test['text'], df_test['reason']])
accuracy(df_test["label"], y_preds)

0.5251111111111111

In [ ]:
classifier_model.save("models/useTripletLoss/")